# Task
We want to achieve the following workflow:

Step 1. 
1. Echo a string to msg.txt 
    
2. Output the length of the string as a number
    
Step 2.
1. Duplicate the content in the file from step 1 two times and redirect it to a new file
    
2. Get the number from step 1 and times the number by 2 and output it

##### In this tutorial, we will complete the task using `PythonOPTemplate` and function OP

import necessary packages

In [1]:
from pathlib import Path

from dflow import Step, Workflow
from dflow.python import OP, Artifact, PythonOPTemplate

For step 1: 

This class can achieve the function to write files. In the example, we try to write a file containing message of string format, and output a number of int format.
- input:
    - "msg": the input message
- output:
    - "out_art": file containing the input message
    - "length": length of the input message

In [2]:
@OP.function
def WriteFile(msg: str) -> {"out_art": Artifact(Path), "length": int}:
    with open("msg.txt","w") as f:
        f.write(msg)
    
    return {
        "out_art": Path("msg.txt"),
        "length": len(msg),
    }

For Step 2:

This OP is to duplicate the content in the file from in_art, and to multiply the input number by 2.

In [3]:
@OP.function
def Duplicate(in_art: Artifact(Path), in_num: int) -> {"out_art": Artifact(Path), "out_num": int}:
    with open(in_art, "r") as f:
        content = f.read()
    with open("bar.txt", "w") as f:
        f.write(content * 2)

    return {
        "out_art": Path("bar.txt"),
        "out_num": in_num * 2,
    }

After defining the PythonOPTemplate, it is to define steps.
- step0 is to write a file and to output length of the content using WriteFile OP and python image
    - parameters:
        - "msg": "HelloWorld!"
- step1 is to duplicate the content in the file and to duplicate the number using Duplicate OP and python image
    - artifacts:
        - in_art file is from step0.outputs.artifacts["out_art"]
    - parameters:
        - in_num is from step0.outputs.parameters["length"]

Finally, we need to set up a Workflow named "python" and then add step0 and step1.

`wf.submit` is to submit this workflow to Argo.

In [4]:
import sys
step0 = Step(
    name="step0",
    template=PythonOPTemplate(WriteFile, image=f"python:{sys.version_info.major}.{sys.version_info.minor}"),
    parameters={"msg": "HelloWorld!"},
)

step1 = Step(
    name="step1",
    template=PythonOPTemplate(Duplicate, image=f"python:{sys.version_info.major}.{sys.version_info.minor}"),
    parameters={"in_num": step0.outputs.parameters["length"]},
    artifacts={"in_art": step0.outputs.artifacts["out_art"]},
)
wf = Workflow(name="python")
wf.add(step0)
wf.add(step1)
wf.submit();

Workflow has been submitted (ID: python-wxj66, UID: 600f99da-21ee-40af-9a3b-c738e46556b8)
